Sarah M. Alghamdi

------------------------------------
# Application of ontology embedding

This is a simple implemantation that takes vectors of genes and diseases and positives dictionary and predict gene-disease association based on unsupervised and supervised methods.

- Unsupervised approach uses cosine similarity

- Supervised method uses MLP with one hidden layer and does the training on 10-fold-cross validation

Using those methods, we can find the most similar gene or genes to diseases inputs :

- genes_vectors_filename : json dictionary {"gene_id":vector of real numbers as a list}

- diseases_vectors_filename : json dictionary {"disease_id":vector of real numbers as a list}

- positives_filename : json dictionary {"disease_id": list of gene ids}

------------------------------------------------------------------------

In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics.pairwise import cosine_similarity
import json
import sys
import numpy as np
import random
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
#cd /content/drive/MyDrive/mowl-tutorial/
import os
os.chdir("/content/drive/MyDrive/Colab_Notebooks/mOWL2/data")

Mounted at /content/drive/


Test files can be downloaded from here:

https://drive.google.com/drive/folders/1_z3-7dhZdF7MbIqDa2T1q4wMzZ809Db_?usp=sharing

These are embeddings generated using DL2Vec tool, on mouse phenotypes

In [2]:
genes_vectors_filename = "data/mouse_genes_embedding.json"
diseases_vectors_filename = "data/human_diseases_embedding.json"
positives_filename = "data/mouse_positives.json"

## 1- Unsupervised Analysis

In this section we calculate the cosine similarity of genes and diseases, then we evaluate the prediction of gene-disease association

In [3]:
with open(genes_vectors_filename,'r') as f:
    genes_vectors = json.load(f)
with open(diseases_vectors_filename,'r') as f:
    diseases_vectors = json.load(f)

with open(positives_filename,'r') as f:
    positives = json.load(f)

human_disease_vectors=[]
human_disease_keys = list(diseases_vectors.keys())
for key in human_disease_keys:
    human_disease_vectors.append(diseases_vectors[key])

mouse_genes_vectors=[]
mouse_genes_keys = list(genes_vectors.keys())
for key in mouse_genes_keys:
    mouse_genes_vectors.append(genes_vectors[key])


Similarity_matrix = cosine_similarity(np.array(human_disease_vectors),np.array(mouse_genes_vectors))

print("the dimentions of this matrix is ", Similarity_matrix.shape)


the dimentions of this matrix is  (12132, 14210)


After calculating cosine similarity between diseases and genes, we then can use these similarities to prdict gene-disease associations

Here we define a function that returns the most similar gene to each disease:

In [4]:
def find_most_similar_gene(disease_id, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    return gene_keys[prediction_list[0]]

def find_top_k_most_similar_genes(disease_id,k, disease_genes_similarity_matrix, disease_keys, gene_keys):
    disease_index = disease_keys.index(disease_id)
    prediction_list = np.flip(np.argsort(disease_genes_similarity_matrix[disease_index]))
    return [gene_keys[prediction_list[x]] for x in range(k)]

print("Testing with the disease (OMIM:106190) Isolated anhidrosis with normal morphology and number sweat glands (ANHD)")

top = find_most_similar_gene("OMIM:106190", Similarity_matrix, human_disease_keys, mouse_genes_keys )

print("The most similar gene to disease OMIM:106190 is "+ top)

top_5 = find_top_k_most_similar_genes("OMIM:106190",5, Similarity_matrix, human_disease_keys, mouse_genes_keys )

print("The most similar genes to disease OMIM:106190 are "+ " ".join(top_5))

Testing with the disease (OMIM:106190) Isolated anhidrosis with normal morphology and number sweat glands (ANHD)
The most similar gene to disease OMIM:106190 is MGI:99418
The most similar genes to disease OMIM:106190 are MGI:99418 MGI:4821354 MGI:1343498 MGI:2670747 MGI:3029925


## 2- Supervised Analysis

In this section we test simple MLP model for the prediction task

In [5]:
# This method is used to generate negative samples
# input:
# # genes_keys: list of genes identifiers "must match the identifiers in the embeddings files"
# # diseases_keys: list of disease identifiers "must match the identifiers in the embeddings files"
# # positives: in a dictionary form
# # hard: binary setting for the split (if hard then negative genes are only sampled from the gene associated diseases)
# output:
# # negatives: in a dictionary, set of genes for each disease is sampled (ratio * number of positive genes)
# # new_positives: returns clean dictionary of positives diseases and genes where only those with representaions are retrived
# # pos_count
# # neg_count
#
# When data are generated the negative genes are selected in 2 ways: hard choise will select the negative genes from the disease associated genes only,
# not hard when the selection of the genes are from associated and non associated genes.
def generate_negatives(genes_keys, diseases_keys, positives, hard):
	negatives = {}
	new_positives = {}
	pos_count = 0
	neg_count = 0
	disease_associated_genes = set([])
	for disease in positives:
		if (disease in diseases_keys):
			for gene in positives[disease]:
				if(gene in genes_keys):
					if(disease not in new_positives):
						new_positives[disease]=set([])
					pos_count+=1
					disease_associated_genes.add(gene)
					new_positives[disease].add(gene)
	non_disease_associated_genes = set([])
	for gene in genes_keys:
		if gene not in disease_associated_genes:
			non_disease_associated_genes.add(gene)

	#genes can be associated or non associated genes
	if not hard:
		for disease in diseases_keys:
			if disease in positives:
				negatives[disease] = set([])
				for gene in genes_keys:
					neg_count+=1
					negatives[disease].add(gene)

	#genes are only the associated genes
	if hard:
		for disease in diseases_keys:
			if disease in positives:
				negatives[disease] = set([])
				for gene in genes_keys:
					if (gene not in positives[disease]) and gene not in non_disease_associated_genes:
						neg_count+=1
						negatives[disease].add(gene)
						break
	return negatives,new_positives, pos_count, neg_count

In [6]:
def get_input_analysis(genes_vectors_filename, diseases_vectors_filename, positives_filename):
	genes_vectors = {}
	with open(genes_vectors_filename,'r') as f:
		genes_vectors = json.load(f)

	diseases_vectors = {}
	with open(diseases_vectors_filename,'r') as f:
		diseases_vectors = json.load(f)

	positives = {}
	with open(positives_filename,'r') as f:
		positives = json.load(f)

	diseases_keys = list(diseases_vectors.keys())
	genes_keys = list(genes_vectors.keys())

	new_positives={}
	for disease in positives:
		if (disease in diseases_keys):
			for gene in positives[disease]:
				if(gene in genes_keys):
					if(disease not in new_positives):
						new_positives[disease]=set([])
					new_positives[disease].add(gene)

	new_disease_keys = [x for x in diseases_keys if x in new_positives]

	print(len(new_disease_keys), len(genes_keys) , len(new_positives.keys()))

	return new_disease_keys,genes_keys,new_positives

In [7]:
def get_input(genes_vectors_filename, diseases_vectors_filename ,positives_filename, ratio):
	genes_vectors = {}
	with open(genes_vectors_filename,'r') as f:
		genes_vectors = json.load(f)

	diseases_vectors = {}
	with open(diseases_vectors_filename,'r') as f:
		diseases_vectors = json.load(f)

	positives = {}
	with open(positives_filename,'r') as f:
		positives = json.load(f)

	diseases_keys = list(diseases_vectors.keys())
	genes_keys = list(genes_vectors.keys())

	negatives, new_positives, pos_count, neg_count = generate_negatives(genes_keys, diseases_keys, positives, hard)


	# Defining Feature Matrex
	X= np.empty(((ratio+1)*pos_count,Vector_size*2))
	y= np.empty((ratio+1)*pos_count)

	negative_diseases = list(negatives.keys())
	sample_number=0
	for disease in new_positives:
		for gene in new_positives[disease]:
			x = np.concatenate((diseases_vectors[disease],genes_vectors[gene]),axis=0)
			X[sample_number]=x
			y[sample_number]=1
			sample_number+=1


			for i in range(ratio):
				n = random.randint(0,len(negative_diseases))
				n_disease = negative_diseases[n-1]
				n = random.randint(0,len(negatives[n_disease]))
				n_gene = list(negatives[n_disease])[n-1]
				x = np.concatenate((diseases_vectors[n_disease],genes_vectors[n_gene]),axis=0)
				X[sample_number]=x
				y[sample_number]=0
				sample_number+=1
	return X,y

In [8]:
def get_training_folds(genes_vectors_filename, diseases_vectors_filename ,positives,diseases_keys,genes_keys, ratio, fold):
	genes_vectors = {}
	with open(genes_vectors_filename,'r') as f:
		genes_vectors = json.load(f)

	diseases_vectors = {}
	with open(diseases_vectors_filename,'r') as f:
		diseases_vectors = json.load(f)

	start = int(len(diseases_keys)*fold/10)
	end = int(len(diseases_keys)*(fold+1)/10) - 1


	testing_disease_keys = diseases_keys[start:end]
	training_disease_keys = [x for x in diseases_keys if x not in testing_disease_keys]

	print(start,end,len(testing_disease_keys),len(training_disease_keys))

	negatives, new_positives, pos_count, neg_count = generate_negatives(genes_keys, training_disease_keys, positives, hard)


	# Defining Feature Matrex
	X= np.empty(((ratio+1)*pos_count,Vector_size*2))
	y= np.empty((ratio+1)*pos_count)

	negative_diseases = list(negatives.keys())
	sample_number=0

	for disease in new_positives:
		for gene in new_positives[disease]:
			x = np.concatenate((diseases_vectors[disease],genes_vectors[gene]),axis=0)
			X[sample_number]=x
			y[sample_number]=1
			sample_number+=1


			for i in range(ratio):
				n = random.randint(1,len(negative_diseases))
				n_disease = negative_diseases[n-1]
				n = random.randint(1,len(negatives[n_disease]))
				n_gene = list(negatives[n_disease])[n-1]
				x = np.concatenate((diseases_vectors[n_disease],genes_vectors[n_gene]),axis=0)
				X[sample_number]=x
				y[sample_number]=0
				sample_number+=1

	index = 0
	X_test= np.empty((len(testing_disease_keys)*len(genes_keys),Vector_size*2))
	y_test= np.empty(len(testing_disease_keys)*len(genes_keys))
	test_guide = {}
	for disease in testing_disease_keys:
		test_guide[disease] = {}
		for gene in genes_keys:
			test_guide[disease][gene] = index
			x = np.concatenate((diseases_vectors[disease],genes_vectors[gene]),axis=0)
			X_test[index]=x
			if(disease in new_positives):
				if(gene in new_positives[disease]):
					y_test[index]=1
				else:
					y_test[index]=0
			else:
				y_test[index]=0
			index+=1



	return X,y , X_test, y_test, test_guide

In [9]:
hard = False
ratio = 5
Vector_size = 100

disease = []
genes = []

HDs_keys,OGs_keys,positives = get_input_analysis(genes_vectors_filename, diseases_vectors_filename, positives_filename)
OGs_HDs_sim = np.empty((len(HDs_keys),len(OGs_keys)))

for fold in range(10):
	print("-------------starting fold--------------")
	print(fold)
	X_train, y_train, X_test, y_test, test_guid = get_training_folds(genes_vectors_filename, diseases_vectors_filename, positives,HDs_keys, OGs_keys, ratio, fold)

	clf = MLPClassifier(hidden_layer_sizes=(Vector_size,), activation= "logistic", solver = "adam", alpha=0.0001, learning_rate= 'constant',learning_rate_init=0.001, random_state=42, max_iter=500, early_stopping=True).fit(X_train, y_train)
	result = clf.predict_proba(X_test)
	print("filling the results")
	for d in range(0,len(HDs_keys)):
		disease = HDs_keys[d]
		if disease in test_guid:
			for g in range(len(OGs_keys)):
				gene=OGs_keys[g]
				index = test_guid[disease][gene]
				OGs_HDs_sim[d][g] = result[index][1]

print("matrix is ready!")

3731 14210 3731
-------------starting fold--------------
0
0 372 372 3359
filling the results
-------------starting fold--------------
1
373 745 372 3359
filling the results
-------------starting fold--------------
2
746 1118 372 3359
filling the results
-------------starting fold--------------
3
1119 1491 372 3359
filling the results
-------------starting fold--------------
4
1492 1864 372 3359
filling the results
-------------starting fold--------------
5
1865 2237 372 3359
filling the results
-------------starting fold--------------
6
2238 2610 372 3359
filling the results
-------------starting fold--------------
7
2611 2983 372 3359
filling the results
-------------starting fold--------------
8
2984 3356 372 3359
filling the results
-------------starting fold--------------
9
3357 3730 373 3358
filling the results
matrix is ready!


-----------------
Lets now test the supervised prediction

In [10]:
print("Testing with the disease (OMIM:106190) Isolated anhidrosis with normal morphology and number sweat glands (ANHD)")

top = find_most_similar_gene("OMIM:106190", OGs_HDs_sim, HDs_keys, OGs_keys )

print("The most similar gene to disease OMIM:106190 is "+ top)

top_5 = find_top_k_most_similar_genes("OMIM:106190",5, OGs_HDs_sim, HDs_keys, OGs_keys )

print("The most similar genes to disease OMIM:106190 are "+ " ".join(top_5))

Testing with the disease (OMIM:106190) Isolated anhidrosis with normal morphology and number sweat glands (ANHD)
The most similar gene to disease OMIM:106190 is MGI:1923452
The most similar genes to disease OMIM:106190 are MGI:1923452 MGI:1261813 MGI:104744 MGI:3052506 MGI:99605


You can call the evaluate_performance function after predicting the labels using the MLP model. Make sure to replace true_labels and predicted_labels with the appropriate variables containing the true labels and predicted labels, respectively.



---

Loading Data: The code starts by loading gene vectors, disease vectors, and positive associations from JSON files.

Calculating Cosine Similarity: It calculates the cosine similarity between diseases and genes using the gene and disease vectors loaded earlier. This similarity matrix is used in the unsupervised method for predicting gene-disease associations.

Unsupervised Analysis: It defines functions (find_most_similar_gene and find_top_k_most_similar_genes) to find the most similar genes to a given disease based on cosine similarity. Then, it tests the unsupervised approach by finding the most similar gene to a specific disease (OMIM:106190) and the top 5 most similar genes.

Supervised Analysis: It defines functions (generate_negatives, get_input, get_training_folds) to prepare input data and train a supervised MLP classifier for predicting gene-disease associations. It trains the MLP model on 10-fold cross-validation and fills a similarity matrix (OGs_HDs_sim) with the predicted probabilities of associations between genes and diseases.

Evaluation: After training the MLP model, it tests the supervised approach by finding the most similar gene to the disease OMIM:106190 and the top 5 most similar genes.

Performance Evaluation: It includes a function (evaluate_performance) to evaluate the performance of the supervised MLP model using metrics such as precision, recall, and F1-score. However, in the provided code, the evaluation is not directly performed. You need to uncomment the lines where true labels are loaded and predicted labels are obtained, and then call the evaluate_performance function with these labels.